In [73]:
#%% Packages
import torch
import numpy as np
# from tqdm import tqdm
from tqdm.notebook import tqdm_notebook as tqdm
import torch.nn.functional as F
import torch.nn as nn

In [74]:
torch.manual_seed(1)

In [75]:
dataset_file = 'V_228.csv'  # 'Letter.csv' for Letter dataset an 'Spam.csv' for Spam dataset
torch.cuda.set_device(0)

In [76]:
#%% System Parameters
# 1. Mini batch size
mb_size = 128
# 2. Missing rate
p_miss = 0.2
# 3. Hint rate
p_hint = 0
# 4. Loss Hyperparameters
alpha = 10
# 5. Train Rate
train_rate = 0.8

#%% Data

# Data generation
Data = np.loadtxt(dataset_file, delimiter=",",skiprows=1)

# Parameters
No = len(Data)
Dim = len(Data[0,:])

# Hidden state dimensions
H_Dim1 = Dim
H_Dim2 = Dim

# Normalization (0 to 1)
Min_Val = np.zeros(Dim)
Max_Val = np.zeros(Dim)

for i in range(Dim):
    Min_Val[i] = np.min(Data[:,i])
    #print(np.min(Data[:,i]))
    Data[:,i] = Data[:,i] - np.min(Data[:,i])
    Max_Val[i] = np.max(Data[:,i])
    Data[:,i] = Data[:,i] / (np.max(Data[:,i]) + 1e-6)    
    



#%% Missing introducing
p_miss_vec = p_miss * np.ones((Dim,1)) 
   
Missing = np.zeros((No,Dim))

for i in range(Dim):
    A = np.random.uniform(0., 1., size = [len(Data),])
    B = A > p_miss_vec[i]
    Missing[:,i] = 1.*B

    
#%% Train Test Division    
   
idx = np.random.permutation(No)

Train_No = int(No * train_rate)
Test_No = No - Train_No
    
# Train / Test Features
trainX = Data[idx[:Train_No],:]
testX = Data[idx[Train_No:],:]

# Train / Test Missing Indicators
trainM = Missing[idx[:Train_No],:]
testM = Missing[idx[Train_No:],:]

    
# Hint Vector Generation
def sample_M(m, n, p):
    A = np.random.uniform(0., 1., size = [m, n])
    B = A > p
    C = 1.*B
    return C

def xavier_init(size):
    in_dim = size[0]
    xavier_stddev = 1. / np.sqrt(in_dim / 2.)
    return np.random.normal(size = size, scale = xavier_stddev)
   

In [77]:
class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc = nn.Linear(456, 228, bias=True)
        self.fc2 = nn.Linear(228, 228, bias=True)
        self.fc3 = nn.Linear(228, 228, bias=True)
        self.relu = nn.ReLU(True)
        self.sigmoid = nn.Sigmoid()

    def forward(self,  New_X, M):
        x = torch.cat(dim=1, tensors=[New_X, M])
        x = x.float()
        x = self.relu(self.fc(x))
        outputs = self.relu(self.fc2(x))
        outputs = self.relu(self.fc3(outputs))
        return self.sigmoid(outputs)

In [78]:
class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc = nn.Linear(456, 228, bias=True)
        self.fc2 = nn.Linear(228, 228, bias=True)
        self.fc3 = nn.Linear(228, 228, bias=True)
        self.relu = nn.ReLU(True)
        self.sigmoid = nn.Sigmoid()

    def forward(self,New_X, H):
        x = torch.cat(dim=1, tensors=[New_X, H])
        x = x.float()
        x = self.relu(self.fc(x))
        outputs = self.relu(self.fc2(x))
        outputs = self.relu(self.fc3(outputs))
        return self.sigmoid(outputs)

In [79]:
def sample_Z(m, n):
    return np.random.uniform(0., 0.01, size = [m, n])        

# Mini-batch generation
def sample_idx(m, n):
    A = np.random.permutation(m)
    idx = A[:n]
    return idx

In [80]:
generator = Generator()
discriminator = Discriminator()

In [81]:
def discriminator_loss(M, New_X, H):
    # Combine with original data
    G_sample = generator(New_X,M)
    
    Hat_New_X = New_X * M + G_sample * (1-M)

    # Discriminator
    D_prob = discriminator(Hat_New_X, H)

    #%% Loss
    D_loss = -torch.mean(M * torch.log(D_prob + 1e-8) + (1-M) * torch.log(1. - D_prob + 1e-8))
    return D_loss

def generator_loss(X, M, New_X, H):
    #%% Structure
    # Generator
    G_sample = generator(New_X,M)

    # Combine with original data
    Hat_New_X = New_X * M + G_sample * (1-M)

    # Discriminator
    D_prob = discriminator(Hat_New_X, H)

    #%% Loss
    G_loss1 = -torch.mean((1-M) * torch.log(D_prob + 1e-8))
    MSE_train_loss = torch.mean((M * New_X - M * G_sample)**2) / torch.mean(M)

    G_loss = G_loss1 + alpha * MSE_train_loss 

    #%% MSE Performance metric
    MSE_test_loss = torch.mean(((1-M) * X - (1-M)*G_sample)**2) / torch.mean(1-M)
    return G_loss, MSE_train_loss, MSE_test_loss
    
def test_loss(X, M, New_X):
    #%% Structure
    # Generator
    G_sample = generator(New_X,M)

    #%% MSE Performance metric
    MSE_test_loss = torch.mean(((1-M) * X - (1-M)*G_sample)**2) / torch.mean(1-M)
    return MSE_test_loss, G_sample

In [82]:
optimizer_D = torch.optim.Adam(params=generator.parameters(), lr = 1)
optimizer_G = torch.optim.Adam(params=discriminator.parameters(), lr = 1)

In [83]:
 def init_weights(m):
    if type(m) == nn.Linear:
        torch.nn.init.xavier_uniform(m.weight)
        m.bias.data.fill_(0.01)

generator.apply(init_weights)
    
discriminator.apply(init_weights)
    

C:\Users\hoon\anaconda3\envs\torch\lib\site-packages\ipykernel_launcher.py:3: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  This is separate from the ipykernel package so we can avoid doing imports until


Discriminator(
  (fc): Linear(in_features=456, out_features=228, bias=True)
  (fc2): Linear(in_features=228, out_features=228, bias=True)
  (fc3): Linear(in_features=228, out_features=228, bias=True)
  (relu): ReLU(inplace=True)
  (sigmoid): Sigmoid()
)

In [84]:
#%% Start Iterations
for it in tqdm(range(5000)):    
    
    #%% Inputs
    mb_idx = sample_idx(Train_No, mb_size)
    X_mb = trainX[mb_idx,:]  
    
    Z_mb = sample_Z(mb_size, Dim) 
    M_mb = trainM[mb_idx,:]  
    H_mb1 = sample_M(mb_size, Dim, 1-p_hint)
    H_mb = M_mb * H_mb1
    
    New_X_mb = M_mb * X_mb + (1-M_mb) * Z_mb  # Missing Data Introduce
    

    X_mb = torch.tensor(X_mb)
    M_mb = torch.tensor(M_mb)
    H_mb = torch.tensor(H_mb)
    New_X_mb = torch.tensor(New_X_mb)
    
    
    optimizer_D.zero_grad()
    D_loss_curr = discriminator_loss(M=M_mb, New_X=New_X_mb, H=H_mb)
    D_loss_curr.backward()
    optimizer_D.step()
 
    optimizer_G.zero_grad()
    #print(M_mb.shape)
    G_loss_curr, MSE_train_loss_curr, MSE_test_loss_curr = generator_loss(X=X_mb, M=M_mb, New_X=New_X_mb, H=H_mb)
    G_loss_curr.backward()
    optimizer_G.step()    
        
    #%% Intermediate Losses
    if it % 100 == 0:
        print('Iter: {}'.format(it),end='\t')
        print('Train_loss: {:.4}'.format(np.sqrt(MSE_train_loss_curr.item())),end='\t')
        print('Test_loss: {:.4}'.format(np.sqrt(MSE_test_loss_curr.item())))

  0%|          | 0/5000 [00:00<?, ?it/s]

Iter: 0	Train_loss: 0.3209	Test_loss: 0.3182
Iter: 100	Train_loss: 0.3266	Test_loss: 0.3287
Iter: 200	Train_loss: 0.3328	Test_loss: 0.3317
Iter: 300	Train_loss: 0.335	Test_loss: 0.3356
Iter: 400	Train_loss: 0.3326	Test_loss: 0.3273
Iter: 500	Train_loss: 0.3211	Test_loss: 0.3235
Iter: 600	Train_loss: 0.3243	Test_loss: 0.3247
Iter: 700	Train_loss: 0.3292	Test_loss: 0.3307
Iter: 800	Train_loss: 0.3284	Test_loss: 0.3251
Iter: 900	Train_loss: 0.3369	Test_loss: 0.3351
Iter: 1000	Train_loss: 0.3232	Test_loss: 0.3191
Iter: 1100	Train_loss: 0.3281	Test_loss: 0.3314
Iter: 1200	Train_loss: 0.3302	Test_loss: 0.3285
Iter: 1300	Train_loss: 0.3142	Test_loss: 0.3119
Iter: 1400	Train_loss: 0.343	Test_loss: 0.3443
Iter: 1500	Train_loss: 0.3276	Test_loss: 0.329
Iter: 1600	Train_loss: 0.3258	Test_loss: 0.3269
Iter: 1700	Train_loss: 0.3362	Test_loss: 0.3388
Iter: 1800	Train_loss: 0.3317	Test_loss: 0.3329
Iter: 1900	Train_loss: 0.3282	Test_loss: 0.3289
Iter: 2000	Train_loss: 0.3365	Test_loss: 0.3398
Iter: 2

In [18]:
Z_mb = sample_Z(Test_No, Dim) 
M_mb = testM
X_mb = testX
        
New_X_mb = M_mb * X_mb + (1-M_mb) * Z_mb  # Missing Data Introduce

X_mb = torch.tensor(X_mb)
M_mb = torch.tensor(M_mb)
New_X_mb = torch.tensor(New_X_mb)

    
MSE_final, Sample = test_loss(X=X_mb, M=M_mb, New_X=New_X_mb)
        
print('Final Test RMSE: ' + str(np.sqrt(MSE_final.item())))

Final Test RMSE: 0.29478107940255177


In [20]:
imputed_data = M_mb * X_mb + (1-M_mb) * Sample
print("Imputed test data:")
np.set_printoptions(formatter={'float': lambda x: "{0:0.8f}".format(x)})
print(imputed_data.detach().numpy())


Imputed test data:
[[0.85438143 0.50000000 0.70190895 ... 0.90425531 0.88837208 0.88807068]
 [0.24355670 0.50000000 0.50000000 ... 0.84650455 0.67888200 0.71428570]
 [0.28608247 0.14285714 0.86196768 ... 0.79787233 0.62875843 0.74521354]
 ...
 [0.26675257 0.85583223 0.48751835 ... 0.56231002 0.12248062 0.61568606]
 [0.31185567 0.50000000 0.47136563 ... 0.50000000 0.75968991 0.57743853]
 [0.69855952 0.85714285 0.50000000 ... 0.98024315 0.89922479 0.63554758]]


In [21]:
# Normalization (0 to 1)
renomal = imputed_data 

for i in range(Dim):
    renomal[:,i] = renomal[:,i]* (Max_Val[i]+1e-6)
    renomal[:,i] = renomal[:,i]+ Min_Val[i]
    
print(renomal.cpu().detach().numpy())

[[69.60000000 43.15000050 60.10000000 ... 67.80000000 67.00000000
  68.10000000]
 [22.20000000 43.15000050 46.35000050 ... 64.00000000 53.48788986
  56.30000000]
 [25.50000000 15.90000000 71.00000000 ... 60.80000000 50.25491939
  58.40000000]
 ...
 [24.00000000 70.30000000 45.50000000 ... 45.30000000 17.60000000
  49.60508402]
 [27.50000000 43.15000050 44.40000000 ... 41.20000050 58.70000000
  47.00807699]
 [57.50821965 70.40000000 46.35000050 ... 72.80000000 67.70000000
  50.95368120]]
